In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO, StringIO
import os

In [2]:
# def download_and_extract(url, path):
#     request = requests.get(url)
#     with ZipFile(BytesIO(request.content), "r") as file:
#          file.extractall(path)

# urls = ["https://m2.chitanka.info/book/25-andersenovi-prikazki.txt.zip",
#        "https://m2.chitanka.info/book/2317-12-mita-v-bylgarskata-istorija.txt.zip",
#        "https://m2.chitanka.info/book/6548-baba-djado-i-vnuche.txt.zip",
#        "https://m2.chitanka.info/book/6761-vasilisa-prekrasna.txt.zip",
#        "https://m2.chitanka.info/book/1824-atentati-koito-trjabvashe-da-promenjat-s.txt.zip",
#        "https://m2.chitanka.info/book/3103-601-izpitani-gotvarski-retsepti.txt.zip",
#        "https://m2.chitanka.info/book/4388-pet-moralni-eseta.txt.zip",
#        "https://m2.chitanka.info/book/366-ochertsi-za-prestypnija-svjat.txt.zip",
#        "https://m2.chitanka.info/book/1031-hakeri-na-choveshkite-dushi.txt.zip",
#        "https://m2.chitanka.info/book/6938-12-printsipa-na-proizvoditelnostta.txt.zip",
#        "https://m2.chitanka.info/book/5226-aleksandyr-makedonski.txt.zip",
#        "https://m2.chitanka.info/book/6393-bogat-tatko-beden-tatko.txt.zip",
#        "https://m2.chitanka.info/book/6748-edin-den-v-dreven-rim.txt.zip",
#        "https://m2.chitanka.info/book/6006-vikingi-i-skrelingi-v-amerika.txt.zip",
#        "https://m2.chitanka.info/book/1498-diktatorite.txt.zip",
#        "https://m2.chitanka.info/book/4098-izsledvane-na-istorijata.txt.zip",
#        "https://m2.chitanka.info/book/1581-igrata-na-lisitsite.txt.zip",
#        "https://m2.chitanka.info/book/1762-istorija-na-vtorata-svetovna-vojna.txt.zip",
#        "https://m2.chitanka.info/book/5084-razgromyt.txt.zip",
#        "https://m2.chitanka.info/book/8106-usmivka-v-polunosht.txt.zip",
#        "https://m2.chitanka.info/book/1016-7-i-37-chudesa.txt.zip",
#        "https://m2.chitanka.info/book/1453-razpoznavane-i-sybirane-na-bilki.txt.zip",
#        "https://m2.chitanka.info/book/1475-grizhi-za-bebeto-i-deteto.txt.zip",
#        "https://m2.chitanka.info/book/524-az-i-moreto.txt.zip",
#        "https://m2.chitanka.info/book/257-vojnishka-tetradka.txt.zip",
#        "https://m2.chitanka.info/book/1435-biblija-star-zavet.txt.zip",
#        "https://m2.chitanka.info/book/6709-david-i-goliat.txt.zip"]

            
# for url in urls:
#     download_and_extract(url,"./training/")

In [3]:
max_line_len = 50

In [4]:
def read_corpus(path):
    files = os.listdir(path)
    lines = []

    for file in files:
        with open("{}/{}".format(path,file), encoding="utf-8-sig") as file_content:
            new_line = ""
            for line in file_content:
                line = line.strip()
                if(len(new_line) + len(line) > max_line_len):
                    tokens = line.split(" ")
                    
                    for token in tokens:
                        if(len(new_line) + len(token) < max_line_len):
                            new_line +=token + " "
                        else:
                            lines.append(new_line)
                            new_line = token
                else:
                    new_line +=line + " "
                    
                    
    return lines

In [5]:
lines = read_corpus("./training/")

In [6]:
len(lines)

335474

In [7]:
lines[-3:]

['издателствона ЦК на ДКМС Държавна печатница ',
 '„ДимитърБлагоев“ София, 1983  Игор Всеволодович ',
 'Можейко7 и 37 ЧУДЕС Главная редакция восточной ']

In [8]:
input_text = [ line.replace(" ", "") for line in lines]

In [9]:
START_CHAR = "\t"
END_CHAR = "\n"

In [10]:
target_text = [ START_CHAR + line + END_CHAR for line in lines]

In [11]:
target_text[0]

'\tБазил Лидъл Харт История на Втората световна война \n'

In [12]:
input_text[:3]

['БазилЛидълХартИсториянаВторатасветовнавойна',
 'ПредговорКогатоДезмъндФлауър,президентътна',
 'издателство„Касъл“,мепомолиданапиша']

In [13]:
input_ch = set()
for line in input_text:
    for c in line:
        input_ch.add(c)

In [14]:
input_ch = list(sorted(input_ch))
input_ch_len = len(input_ch)
input_ch_len

213

In [15]:
target_ch = set()
for line in target_text:
    for c in line:
        target_ch.add(c)

In [16]:
target_ch_len = len(target_ch)
target_ch_len

215

In [17]:
input_ch_idx = dict([(char,i) for i, char in enumerate(input_ch)])
target_ch_idx = dict([(char,i) for i, char in enumerate(target_ch)])

In [18]:
max_input_len = max([len(line) for line in input_text])
max_target_len = max([len(line) for line in target_text])

print(max_input_len)
print(max_target_len)

280
282


In [19]:
import numpy as np

In [20]:
def encode(input_samples=[], target_samples=[]):
    
    if len(input_samples) != len(target_samples):
        print("INPUT AND TARGET have diff size")
        return None
    
    max_input_len = max([len(sample) for sample in input_samples])
    max_target_len = max([len(sample) for sample in target_samples])

    # Тук [This index is for sample] [This index is for char in sample][This index is one hot encoding the char]
    
    encoder_input = np.zeros((len(input_samples), max_input_len, input_ch_len),dtype="float64")
    decoder_input = np.zeros((len(input_samples), max_target_len, target_ch_len),dtype="float64")
    decoder_output = np.zeros((len(input_samples), max_target_len, target_ch_len),dtype="float64")
    
    
    for i, (input_sample, target_sample) in enumerate(zip(input_samples, target_samples)):
        for t, char in enumerate(input_sample):
            encoder_input[i, t, input_ch_idx[char]] = 1.
        for t, char in enumerate(target_sample):
            decoder_input[i, t, target_ch_idx[char]] = 1.
            if t > 0:
                decoder_output[i, t - 1, target_ch_idx[char]] = 1.
            
    return encoder_input, decoder_input, decoder_output

In [21]:
import keras
from keras.models import Model
from keras.layers import Input,Dense,LSTM, Embedding, Dropout

Using TensorFlow backend.


In [22]:
STATE_SPACE_DIM = 128

In [23]:
encoder_inputs = Input(shape=(None, input_ch_len))
encoder = LSTM(STATE_SPACE_DIM, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [24]:
decoder_inputs = Input(shape=(None, target_ch_len))
dropout = Dropout(0.2)(decoder_inputs)
decoder_lstm = LSTM(STATE_SPACE_DIM, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dropout,
                                     initial_state=encoder_states)
decoder_dense = Dense(target_ch_len, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [25]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='adam', loss='categorical_crossentropy')

In [26]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, 215)    0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None, 213)    0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, None, 215)    0           input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 128), (None, 175104      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LS

In [27]:
 import gc


In [28]:
batch_size = 30000
cursor = 0
while cursor+batch_size < len(input_text):
    gc.collect()
    encoder_input_data, decoder_input_data, decoder_target_data = encode(input_samples=input_text[cursor:cursor+batch_size],
                                                                    target_samples=target_text[cursor:cursor+batch_size])
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=256,
          epochs=10,
          validation_split=0.2)
    cursor+=batch_size
    print(cursor)
    del encoder_input_data,decoder_input_data, decoder_target_data
    gc.collect()
    
gc.collect()
encoder_input_data, decoder_input_data, decoder_target_data = encode(input_samples=input_text[cursor:],
                                                                    target_samples=target_text[cursor:])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=256,
          epochs=10,
          validation_split=0.2)


Train on 24000 samples, validate on 6000 samples
Epoch 1/10
24000/24000 [==============================] - 11s 470us/step - loss: 3.1263 - val_loss: 2.775432 - ETA: 0s - loss: 3.129
Epoch 2/10
24000/24000 [==============================] - 10s 428us/step - loss: 2.7407 - val_loss: 2.6888
Epoch 3/10
24000/24000 [==============================] - 10s 428us/step - loss: 2.6446 - val_loss: 2.5730TA: 
Epoch 4/10
24000/24000 [==============================] - 10s 428us/step - loss: 2.5381 - val_loss: 2.4654
Epoch 5/10
24000/24000 [==============================] - 10s 430us/step - loss: 2.4560 - val_loss: 2.3803
Epoch 6/10
24000/24000 [==============================] - 10s 432us/step - loss: 2.3956 - val_loss: 2.3166
Epoch 7/10
24000/24000 [==============================] - 10s 429us/step - loss: 2.3487 - val_loss: 2.2661
Epoch 8/10
24000/24000 [==============================] - 10s 431us/step - loss: 2.3191 - val_loss: 2.2297- ETA: 0s - loss: 
Epoch 9/10
24000/24000 [=======================

24000/24000 [==============================] - 11s 439us/step - loss: 1.7083 - val_loss: 1.5141
Epoch 4/10
24000/24000 [==============================] - 11s 439us/step - loss: 1.6928 - val_loss: 1.5047
Epoch 5/10
24000/24000 [==============================] - 10s 437us/step - loss: 1.6781 - val_loss: 1.4898
Epoch 6/10
24000/24000 [==============================] - 11s 439us/step - loss: 1.6657 - val_loss: 1.4737
Epoch 7/10
24000/24000 [==============================] - 11s 438us/step - loss: 1.6545 - val_loss: 1.4581
Epoch 8/10
24000/24000 [==============================] - 11s 438us/step - loss: 1.6436 - val_loss: 1.4585
Epoch 9/10
24000/24000 [==============================] - 11s 449us/step - loss: 1.6338 - val_loss: 1.4396
Epoch 10/10
24000/24000 [==============================] - 11s 455us/step - loss: 1.6228 - val_loss: 1.4306
240000
Train on 24000 samples, validate on 6000 samples
Epoch 1/10
24000/24000 [==============================] - 12s 482us/step - loss: 1.6371 - val_loss

In [36]:
model.save("more_more_data.h5")

/usr/local/lib/python3.5/dist-packages/keras/engine/topology.py:2344: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 128) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [30]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(STATE_SPACE_DIM,))
decoder_state_input_c = Input(shape=(STATE_SPACE_DIM,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


In [31]:
reverse_input_ch_idx = dict(
    (i, char) for char, i in input_ch_idx.items())
reverse_target_ch_idx= dict(
    (i, char) for char, i in target_ch_idx.items())

In [32]:
def decode_sequence(input_seq):
  
    states_value = encoder_model.predict(input_seq)

  
    target_seq = np.zeros((1, 1, len(target_ch)))
   
    target_seq[0, 0, target_ch_idx[START_CHAR]] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_ch_idx[sampled_token_index]
        decoded_sentence += sampled_char

        
        if (sampled_char == END_CHAR or
           len(decoded_sentence) > max_target_len):
            stop_condition = True

       
        target_seq = np.zeros((1, 1, len(target_ch)))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence

In [33]:
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_text[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: БазилЛидълХартИсториянаВторатасветовнавойна
Decoded sentence: ченатой де и полиго — кура си града, както се да 

-
Input sentence: ПредговорКогатоДезмъндФлауър,президентътна
Decoded sentence: указазарава на на прединстта на строите на 

-
Input sentence: издателство„Касъл“,мепомолиданапиша
Decoded sentence: Призсъв на граната и се на върха и да се извечен 

-
Input sentence: предговоракъмкнигатанасъпругами„Историяна
Decoded sentence: историята са да се нагуда — в Борма на Бура се са 

-
Input sentence: Вторатасветовнавойна“,многоскоросидадох
Decoded sentence: хълмалкоредвели на голата с проди. И да се 

-
Input sentence: сметка,чезадаблагодарянавсички,които
Decoded sentence: селитавали въще ст полали на била преди са на 

-
Input sentence: помогналивподготовкатапонаписванетой,трябва
Decoded sentence: редитекатели на преди. Простава пред Ангура. И да 

-
Input sentence: даотдамзаслуженопризнаниенастотицихора,от
Decoded sentence: Ванрайна.Кога обил бил да ставал на стро

-
Input sentence: ипопо-широккръгвъпроси.Естествено,тезимои
Decoded sentence: чеамасъщи да деле и да страната към това. Това се 

-
Input sentence: запискинебихамоглидаосветлятнапълно
Decoded sentence: Катоба от сирови и подарва в краните на синия 

-
Input sentence: достовернотова,коетотесамислелипрединякои
Decoded sentence: познавана нариза. По толко се съде какво си 

-
Input sentence: конкретнисъбитияирешения,новсепакса
Decoded sentence: никоикара и и са послените на страната на сери 

-
Input sentence: направенипредиизлизанетонамемоарите,замъглени
Decoded sentence: сдрагости пъте. Ази по пред 1941 годана на Акара, 

-
Input sentence: доизвестнастепенотизминалитегодини,но
Decoded sentence: наначастници за нашили се на съветата страните 

-
Input sentence: технитеразказимогатдабъдатсъпоставенис
Decoded sentence: пробелиза запазвани на се изглавал с 

-
Input sentence: разказитенадругисвидетелинасъщитесъбития,
Decoded sentence: комнада повени на волите на Буд пред Бермани и 

-
Input

In [34]:
def encode(input_name):
    encoded = np.zeros (( 1 , max_input_len, input_ch_len ),dtype="float64" )
    for t, char in enumerate(input_name):
        encoded[0, t, input_ch_idx[char]] = 1.
    return encoded[0:1]

In [35]:
print(decode_sequence(encode("дасмеживииздрави")))
print(decode_sequence(encode("замногогодини")))

саййше не жило пакула за Мал — 

байййногожели от ула, и пазу — 

